# SPOT Introduction and basic functions

In [ ]:
import bosdyn.client
from bosdyn.client.image import ImageClient
from bosdyn.client.point_cloud import PointCloudClient
from bosdyn.geometry import EulerZXY
from bosdyn.client import math_helpers
from bosdyn.client.robot_command import RobotCommandClient, RobotCommandBuilder, blocking_stand, blocking_sit, blocking_selfright
from bosdyn.client.frame_helpers import BODY_FRAME_NAME, ODOM_FRAME_NAME, VISION_FRAME_NAME, get_se2_a_tform_b
from bosdyn.api.basic_command_pb2 import RobotCommandFeedbackStatus
from bosdyn.api import robot_state_pb2

import time
import numpy as np

In [ ]:
# Create SDK object to access basic methods
sdk = bosdyn.client.create_standard_sdk('demo')

In [ ]:
# Create robot object with IP address/Hostname as argument
robot = sdk.create_robot('192.168.80.3')

# Create id_client object
try:
    id_client = robot.ensure_client('robot-id')
except Exception as e:
    print(f'Could not access ID client. Check network connection. Error: {e}')

# Retrieve robot id (blocking function, timeout can be set in seconds)
spot_id = id_client.get_id(timeout=5)
print(f'Client ID: {spot_id}')

### Authenticate

In [ ]:
# Many services require authentication before they can be used, like the robotStateClient
try:
    robot.authenticate('user', 'fntivr1zdj7w')

except Exception as e:
    print(f'Could not authenticate. Error: {e}')
    exit()

### State client for robot status messages

In [ ]:
# Create a state client object to gain access to status messages
state_client = robot.ensure_client('robot-state')

# Prints a status message of the state of the robot
print(state_client.get_robot_state())

In [ ]:
# Access single properties like so: 
state_client.get_robot_state().battery_states[0]

### Estop client for controlling motor power 

In [ ]:
# Create E-stop client object to be able to enable/disable the motor power
estop_client = robot.ensure_client('estop')

# Print current estop state
print(estop_client.get_status())

In [ ]:
# Create an estop endpoint to check regulary if the robot is controlled safely. If the timeout is exceeded, it cuts motor power
estop_endpoint = bosdyn.client.estop.EstopEndpoint(client=estop_client, name='my_estop', estop_timeout=9.0)
estop_endpoint.force_simple_setup()

# The estop_keep_alive object is called on a regular basis in the background to check in with the endpoint.
estop_keep_alive = bosdyn.client.estop.EstopKeepAlive(estop_endpoint)

# prints out the current estop status. Stop_level: ESTOP_LEVEL_NONE indicates that the estop can be started
print(estop_client.get_status())

### Lease client for accessing motor control authority

In [ ]:
# To aquire authority over Spots motors, the operator has to take ownership
# This prevents multiple sessions to access the control
# Every command to the robot has to be issued with a valid lease
# After use, the lease can be returned
lease_client = robot.ensure_client('lease')

# Lists the current leases
print(lease_client.list_leases())

try:
    # Creating a lease object with the lease client
    lease = lease_client.acquire()

except:
    # If lease is already aquired, take it forcefully
    lease = lease_client.take()

# Check in with the robot
lease_keep_alive = bosdyn.client.lease.LeaseKeepAlive(lease_client)

### Powering the motors

In [ ]:
try:
# Now the robot can be powered on
    robot.power_on(timeout_sec=20)
    
except Exception as e:
    print(e)

# Check the power state
print(f'Robot is powered: {robot.is_powered_on()}')

# Timesync assures that the controlling device and the robot are synchronised whichis important for time critical functions 
robot.time_sync.wait_for_sync()


### Controlling the robot

In [ ]:
# Create a command client object to issue commands to the robot
command_client = robot.ensure_client(RobotCommandClient.default_service_name)

# This command lets the robot stand up
blocking_stand(command_client, timeout_sec=10)


### Creating move commands

In [ ]:
# Prepares a frame in which the robot turns around the X axis
footprint_R_body = EulerZXY(yaw=-0.5, roll=0.2, pitch=-0.4)

# The footprint is then given to the command builder
cmd = RobotCommandBuilder.synchro_stand_command(body_height=0.1, footprint_R_body=footprint_R_body)

# Ahe resulting command "cmd" is then passed to the command service
command_client.robot_command(cmd)

In [ ]:
# A new command is created to raise the robot up, overwriteing the previous command
cmd = RobotCommandBuilder.synchro_sit_command()
command_client.robot_command(cmd)

In [ ]:
# A new command is created to raise the robot up, overwriteing the previous command
cmd = RobotCommandBuilder.synchro_stand_command(body_height=0.1)
command_client.robot_command(cmd)

In [ ]:
# Function to move robot relative to its current position
def relative_move(dx, dy, dyaw, frame_name, robot_command_client, robot_state_client, stairs=False):

    # Gets snapshot of its current position    
    transforms = robot_state_client.get_robot_state().kinematic_state.transforms_snapshot

    # Build the transform for where we want the robot to be relative to where the body currently is.
    body_tform_goal = math_helpers.SE2Pose(x=dx, y=dy, angle=dyaw)

    # We do not want to command this goal in body frame because the body will move, thus shifting
    # our goal. Instead, we transform this offset to get the goal position in the output frame
    # (which will be either odom or vision).

    out_tform_body = get_se2_a_tform_b(transforms, frame_name, BODY_FRAME_NAME)
    out_tform_goal = out_tform_body * body_tform_goal

    # Command the robot to go to the goal point in the specified frame. The command will stop at the
    # new position.
    robot_cmd = RobotCommandBuilder.synchro_se2_trajectory_point_command(
        goal_x=out_tform_goal.x, goal_y=out_tform_goal.y, goal_heading=out_tform_goal.angle,
        frame_name=frame_name, params=RobotCommandBuilder.mobility_params(stair_hint=stairs))
    
    end_time = 10.0
    cmd_id = robot_command_client.robot_command(lease=None, command=robot_cmd,
                                                end_time_secs=time.time() + end_time)
    # Wait until the robot has reached the goal.
    while True:
        feedback = robot_command_client.robot_command_feedback(cmd_id)
        mobility_feedback = feedback.feedback.synchronized_feedback.mobility_command_feedback
        if mobility_feedback.status != RobotCommandFeedbackStatus.STATUS_PROCESSING:
            print("Failed to reach the goal")
            return False
       
        traj_feedback = mobility_feedback.se2_trajectory_feedback
        if (traj_feedback.status == traj_feedback.STATUS_AT_GOAL and
                traj_feedback.body_movement_status == traj_feedback.BODY_STATUS_SETTLED):
            print("Arrived at the goal.")
            return True

In [ ]:
relative_move(dx=1, dy=0, dyaw=0, frame_name = ODOM_FRAME_NAME, robot_command_client=command_client, robot_state_client=state_client, stairs=False)

### Powering off

In [ ]:

# The cut_immediately argument can be set to false to sit the robot down gracefully without crashing into the floor
robot.power_off(cut_immediately=False)

robot.time_sync.stop()
